In [3]:
import cv2
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

Setting up Selenium WebDriver

In [5]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get("ANY GAME URL")
# https://k4.games/play/neon-rider/    (this is just a example of URL, this program doesn't have the logic of game)

Prepareing the input image by converting it to grayscale.

Applying edge detection to the preprocessed image to highlight the edges.

In [6]:
def capture_screen():
    screenshot = driver.get_screenshot_as_png()
    image = np.frombuffer(screenshot, dtype='uint8')
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (84, 84))
    normalized = resized / 255.0
    normalized = (normalized * 255).astype(np.uint8)
    return normalized
# Resize the image of game so that i will be easy to make decision easy.
def interpret_game_state(image):
    edges = cv2.Canny(image, threshold1=100, threshold2=200)
    return edges
    

LOGIC FOR THE GAME. 

IT MAY BE ANY GAME AND IT'S LOGIC

In [7]:
def make_decision(game_state):
    if np.sum(game_state[:, :42]) > np.sum(game_state[:, 42:]):
        return 'move_right'
    else:
        return 'move_left'

def perform_action(action):
    body = driver.find_element(By.TAG_NAME, 'body')
    if action == 'move_right':
        body.send_keys(Keys.ARROW_RIGHT)
    elif action == 'move_left':
        body.send_keys(Keys.ARROW_LEFT)

MAIN LOOP

In [8]:
while True:
    screen = capture_screen()
    preprocessed_image = preprocess_image(screen)
    game_state = interpret_game_state(preprocessed_image)
    action = make_decision(game_state)
    perform_action(action)
    time.sleep(0.1)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.183)
Stacktrace:
	GetHandleVerifier [0x00EEC203+27395]
	(No symbol) [0x00E83E04]
	(No symbol) [0x00D81B7F]
	(No symbol) [0x00D5E483]
	(No symbol) [0x00DEA06F]
	(No symbol) [0x00DFC3D6]
	(No symbol) [0x00DE3736]
	(No symbol) [0x00DB7541]
	(No symbol) [0x00DB80BD]
	GetHandleVerifier [0x011A3AB3+2876339]
	GetHandleVerifier [0x011F7F7D+3221629]
	GetHandleVerifier [0x00F6D674+556916]
	GetHandleVerifier [0x00F7478C+585868]
	(No symbol) [0x00E8CE44]
	(No symbol) [0x00E89858]
	(No symbol) [0x00E899F7]
	(No symbol) [0x00E7BF4E]
	BaseThreadInitThunk [0x768C7BA9+25]
	RtlInitializeExceptionChain [0x776CC10B+107]
	RtlClearBits [0x776CC08F+191]
